In [ ]:
"""
Phase 1 of the Project

Building a mixed-integer linear programming model to find any feasible 
solution to the scheduling problem, without a clear objective function yet, but
subject to the following constraints:

1) Coverage Requirements: Each shift j must be covered by 
at least Bjk employees who have skill k.

2) One job per employee per shift: Each employee i can work at most one job (skill k) 
per shift j.

3) Skill Eligibility and Availability: Each employee i can only be assigned to a given
shift & station jk if they have the appropriate skill k, and are available to work at that shift j.

4) Shift Length: Each employee must work at least a specificed number of shifts, and under a specified maximum number of shifts.
"""

In [18]:
import random
# Parameters

# Number of employees
n = 20

# Number of shifts
m = 10

# Number of stations/skills
k = 4

# Number of shifts each employee must work
min_shifts = 5

# Maximum number of shifts each employee can work
max_shifts = 10

# Maximum number of stations an employee can work during a given shift
max_stations_per_shift = 1

# Name of stations
stations = ["Omlette", "Grill", "Stir Fry", "Prep"]

# Name of shifts
shifts = [num for num in range(1, m+1)]

#Name of Employees 
employees = ["Alice", "Bob", "Charlie", "David", "Eve", "Frank", "Grace", "Hannah", "Ivy", "Jack", "Kathy", "Liam", "Mia", "Nathan", "Olivia", "Peter", "Quinn", "Rachel", "Sam", "Tom"]

"""
Number of employees required for each shift and station
Each key in the dictionary is a skill, shift number tuple, and the value is the number of employees required
For example, B[1, 1] = 2 means that 2 employees at shift 1 are needed for station 1 (Omlette)
""" 

B = {(1,1):0, (1,2):1, (1,3):0, (1,4):1, (2,1): 0, (2,2): 0, (2,3): 0, (2,4): 0, (3,1): 0, (3,2): 0, (3,3): 1, (3,4): 2, (4,1): 0, (4,2): 0, (4,3): 1, (4,4): 1, (5,1): 1, (5,2): 1, (5,3): 0, (5,4): 0, (6,1): 1, (6,2): 2, (6,3): 1, (6,4): 0, (7,1): 0, (7,2): 1, (7,3): 1, (7,4): 1, (8,1): 1, (8,2): 0, (8,3): 0, (8,4): 2, (9,1): 1, (9,2): 1, (9,3): 1, (9,4): 0, (10,1): 1, (10,2): 0, (10,3): 0, (10,4): 0}
# Transform the keys: map station number to station name.
B = {(shift, stations[station_num - 1]): value for (shift, station_num), value in B.items()}

"""
Stations each employee can work at
Each key in the dictionary is an employee name, and the value is a list of stations they can work at
For example, S["Alice"] = [1, 2] means that Alice can work at stations 1 and 2
"""
S = {employees[i]: stations for i in range(n)}


"""
Employees who are available for each shift
Each key in the dictionary is a shift number, and the value is a list of employees who are available for that shift
For example, A[1] = ["Alice", "Bob"] means that Alice and Bob are available for shift 1
"""
employee_availability = {
    employee: sorted(random.sample(shifts, random.randint(7, 10)))
    for employee in employees
}

# Build the A dictionary: for each shift, list the employees available
A = {shift: [] for shift in shifts}
for employee, available_shifts in employee_availability.items():
    for shift in available_shifts:
        A[shift].append(employee)
#A = {j: employees for j in shifts}

In [2]:
import gurobipy as gp
from gurobipy import *
from gurobipy import Model

In [24]:
model = Model("Shift Scheduling")

# Decision Variables
# x[i,j,k] = 1 if employee i is assigned to shift j at station k, 0 otherwise
x = {}
for employee_i in employees: 
    for shift_j in shifts:
        if employee_i in A[shift_j]:
            for station_k in stations:
                if station_k in S[employee_i]:
                    x[employee_i, shift_j, station_k] = model.addVar(vtype=GRB.BINARY, name=f"x[{employee_i},{shift_j},{station_k}]")

model.update()

# Objective Function (for now, just find any solution)
c_penalty = 1
model.setObjective(
    c_penalty * quicksum(
        x[employee_i, shift_j, station_k]
        for employee_i in employees
        for shift_j in shifts 
        for station_k in stations
        if (employee_i, shift_j, station_k) in x
    ),
    sense = GRB.MINIMIZE
)

# Constraint on coverage requirements
model.addConstrs(
    quicksum(
        x[employee_i, shift_j, station_k]
        for employee_i in employees
        if (employee_i, shift_j, station_k) in x
    ) >= B[shift_j, station_k]
    for shift_j in shifts
    for station_k in stations                                          
    if (shift_j, station_k) in B
)

# Constraint on one job per employee per shift
model.addConstrs(
    quicksum(
        x[employee_i, shift_j, station_k]
        for station_k in stations
        if (employee_i, shift_j, station_k) in x
    ) <= max_stations_per_shift
    for employee_i in employees
    for shift_j in shifts
)

# Constraint on skill eligibility and availability
# model.addConstrs(
#     x[employee_i, shift_j, station_k] == 0
#     for employee_i in employees
#     for shift_j in shifts
#     for station_k in stations
#     if employee_i not in A[shift_j] or station_k not in S[employee_i]
# )

# Constraint on shift length -- >= min_shifts
model.addConstrs(
    min_shifts <= quicksum(
        x[employee_i, shift_j, station_k]
        for shift_j in shifts
        for station_k in stations
        if (employee_i, shift_j, station_k) in x
    ) 
    for employee_i in employees

)


# Constraint on shift length -- <= max_shifts

model.addConstrs(
    quicksum(
        x[employee_i, shift_j, station_k]
        for shift_j in shifts
        for station_k in stations
        if (employee_i, shift_j, station_k) in x
    ) <= max_shifts
    for employee_i in employees

)

{'Alice': <gurobi.Constr *Awaiting Model Update*>,
 'Bob': <gurobi.Constr *Awaiting Model Update*>,
 'Charlie': <gurobi.Constr *Awaiting Model Update*>,
 'David': <gurobi.Constr *Awaiting Model Update*>,
 'Eve': <gurobi.Constr *Awaiting Model Update*>,
 'Frank': <gurobi.Constr *Awaiting Model Update*>,
 'Grace': <gurobi.Constr *Awaiting Model Update*>,
 'Hannah': <gurobi.Constr *Awaiting Model Update*>,
 'Ivy': <gurobi.Constr *Awaiting Model Update*>,
 'Jack': <gurobi.Constr *Awaiting Model Update*>,
 'Kathy': <gurobi.Constr *Awaiting Model Update*>,
 'Liam': <gurobi.Constr *Awaiting Model Update*>,
 'Mia': <gurobi.Constr *Awaiting Model Update*>,
 'Nathan': <gurobi.Constr *Awaiting Model Update*>,
 'Olivia': <gurobi.Constr *Awaiting Model Update*>,
 'Peter': <gurobi.Constr *Awaiting Model Update*>,
 'Quinn': <gurobi.Constr *Awaiting Model Update*>,
 'Rachel': <gurobi.Constr *Awaiting Model Update*>,
 'Sam': <gurobi.Constr *Awaiting Model Update*>,
 'Tom': <gurobi.Constr *Awaiting Mode

In [20]:
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 7730U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 280 rows, 712 columns and 2848 nonzeros
Model fingerprint: 0x93ae5e49
Variable types: 0 continuous, 712 integer (712 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 100.0000000
Presolve removed 95 rows and 334 columns
Presolve time: 0.01s
Presolved: 185 rows, 378 columns, 1082 nonzeros
Variable types: 0 continuous, 378 integer (378 binary)

Root relaxation: cutoff, 20 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     

In [22]:
sum = 0
for v in model.getVars():
    if v.X != 1.0:
        sum += 1
    print(f"{v.VarName}: {v.X}")
print(sum)

x[Alice,2,Omlette]: 0.0
x[Alice,2,Grill]: 1.0
x[Alice,2,Stir Fry]: 0.0
x[Alice,2,Prep]: 0.0
x[Alice,3,Omlette]: 0.0
x[Alice,3,Grill]: 0.0
x[Alice,3,Stir Fry]: 0.0
x[Alice,3,Prep]: 1.0
x[Alice,4,Omlette]: 0.0
x[Alice,4,Grill]: 0.0
x[Alice,4,Stir Fry]: 0.0
x[Alice,4,Prep]: 0.0
x[Alice,5,Omlette]: 1.0
x[Alice,5,Grill]: 0.0
x[Alice,5,Stir Fry]: 0.0
x[Alice,5,Prep]: 0.0
x[Alice,6,Omlette]: 0.0
x[Alice,6,Grill]: 1.0
x[Alice,6,Stir Fry]: 0.0
x[Alice,6,Prep]: 0.0
x[Alice,7,Omlette]: 0.0
x[Alice,7,Grill]: 0.0
x[Alice,7,Stir Fry]: 0.0
x[Alice,7,Prep]: 0.0
x[Alice,8,Omlette]: 0.0
x[Alice,8,Grill]: 0.0
x[Alice,8,Stir Fry]: 0.0
x[Alice,8,Prep]: 1.0
x[Bob,1,Omlette]: 0.0
x[Bob,1,Grill]: 0.0
x[Bob,1,Stir Fry]: 0.0
x[Bob,1,Prep]: 1.0
x[Bob,2,Omlette]: 0.0
x[Bob,2,Grill]: 0.0
x[Bob,2,Stir Fry]: 1.0
x[Bob,2,Prep]: 0.0
x[Bob,3,Omlette]: 0.0
x[Bob,3,Grill]: 0.0
x[Bob,3,Stir Fry]: 1.0
x[Bob,3,Prep]: 0.0
x[Bob,4,Omlette]: 0.0
x[Bob,4,Grill]: 0.0
x[Bob,4,Stir Fry]: 0.0
x[Bob,4,Prep]: 1.0
x[Bob,5,Omlette]: 0.